In [2]:
import requests
from lxml import etree
from tqdm.notebook import tqdm
from time import time
# import pinyin
import pypinyin
from pypinyin import pinyin,lazy_pinyin
import jieba
import re

In [3]:
entity = requests.post("http://robotsit.suning.com/skill/queryWordEntity").json()['data']

with open(r'D:\Users\Desktop\slot_hanzi.txt','w',encoding='utf8') as f:
    for e in entity:
        f.write(e['entityName']+"\n")
        # print(e)
        # break

In [4]:
s = time()
def py(txt):
    return pinyin.get(txt, format="strip", delimiter="")

def py_withspace(txt,heteronym=True):
    py_result = pinyin(txt, heteronym=heteronym,style=0)
    # result = [[] for i in range(len(py_result))]
    # for idx, ii in  enumerate(py_result):
    #     for i in ii:
    #         if len(i)>1 and i[-1] in ['1','2','3','4']:
    #             result[idx].append(i[:-1])
    #         else:
    #             result[idx].append(i)

    return cat_list_py(py_result)

def py_lazy(txt):
    return " ".join(lazy_pinyin(txt))

digital_dict={'1':'一','2':'二','3':'三',"4":'四','5':'五',"6":"六","7":"七","8":"八","9":"九","0":"零"}

def dig_convert(txt):
    result = ''
    for x in txt:
        result+=digital_dict.get(x,x)
    return result
print(dig_convert('1243'))
print(py_withspace('B201七件事'),py_withspace('事'))
print((time()-s)*1000)

一二四三


NameError: name 'cat_list_py' is not defined

In [5]:
def pinyin_aug_1(txt):
    """
    zh,ch,sh - > z,c,s...
    """
    if 'zh' in txt:
        txt = txt.replace('zh','z')
    if 'ch' in txt:
        txt = txt.replace('ch','c')
    if 'sh' in txt:
        txt = txt.replace('sh','s')
    return txt
    
def pinyin_aug_11(txt):
    if 'ing' in txt:
        txt = txt.replace('ing','in')
    if 'ang' in txt:
        txt = txt.replace('ang','an')
    if 'eng' in txt:
        txt = txt.replace('eng','en')
    if 'ong' in txt:
        txt = txt.replace('ong','un')
    return txt

def pinyin_aug_2(txt):
    """
    z,c,s -> zh,ch,sh ...
    """
    if 'z' in txt and 'zh' not in txt:
        txt = txt.replace('z','zh')
    if 'c' in txt and 'ch' not in txt:
        txt = txt.replace('c','ch')
    if 's' in txt and 'sh' not in txt:
        txt = txt.replace('s','sh')
    return txt

def pinyin_aug_22(txt):
    if 'in' in txt and 'ing' not in txt:
        txt = txt.replace('in','ing')
    if 'an' in txt and 'ang' not in txt:
        txt = txt.replace('an','ang')
    if 'en' in txt and 'eng' not in txt:
        txt = txt.replace('en','eng')
    if 'un' in txt and 'ong' not in txt :
        txt = txt.replace('un','ong')
    return txt

def pinyin_aug_3(txt):
    """
    l<-->n
    """
    if txt.startswith('l'):
        txt = txt.replace('l','n')
    elif txt.startswith('n'):
        txt = txt.replace('n','l')
    return txt

In [6]:
## 拼音增强方法
def cat_list_py(list_py):
    result = []
    count = 0
    ss =''
    def func(list_py, count, result, ss):
        cur_list= list_py[count]
        for cur in cur_list:
            if count+1  < len(list_py):
                func(list_py, count+1 , result , ss + " "+cur)
            else:
                result.append(ss + " "+cur)
    func(list_py, count, result, ss)
    return [r.strip() for r in result]

cat_list_py([['jie','ji'], ['jia','1','2'], ['ri','3']])

['jie jia ri',
 'jie jia 3',
 'jie 1 ri',
 'jie 1 3',
 'jie 2 ri',
 'jie 2 3',
 'ji jia ri',
 'ji jia 3',
 'ji 1 ri',
 'ji 1 3',
 'ji 2 ri',
 'ji 2 3']

In [32]:
### 获取所有词槽的拼音及变形
from tqdm.notebook import tqdm
re_sub = lambda x:"".join(re.findall('[\u4e00-\u9fa5a-zA-Z0-9]+',x))
def create_han_pin_dict(entity):
    han_pinyin = {}
    han_pinyin_reverse = {}
    for e in tqdm(entity):
        name = re_sub(e['entityName'])
        py_lists = py_withspace(name)
        for py_list in py_lists:
            py_list_size = len(py_list.split())
            if py_list_size>=6 or py_list_size<=1:
                han_pinyin.setdefault(name,[]).append(py_list)
                han_pinyin_reverse.setdefault(py_list.lower(),[]).append(name)
            else:
                list_py = [[] for i in range(py_list_size)]
                for idx, word in enumerate(py_list.split()):
                    list_py[idx].append(pinyin_aug_1(word))
                    list_py[idx].append(pinyin_aug_11(word))
                    list_py[idx].append(pinyin_aug_2(word))
                    list_py[idx].append(pinyin_aug_22(word))
                    list_py[idx].append(pinyin_aug_3(word))
                for pin in set(cat_list_py(list_py)):
                    han_pinyin.setdefault(name,[]).append(pin)
                    han_pinyin_reverse.setdefault(pin.lower(),[]).append(name)
    return han_pinyin,han_pinyin_reverse

In [35]:
han_pinyin_reverse['b101']

['B101']

In [9]:
x = 'chen'
pinyin_aug_1(x),'|||',pinyin_aug_2(x),'||',pinyin_aug_3(x)

('cen', '|||', 'chen', '||', 'chen')

In [19]:
han_pinyin_reverse['b101'.lower()]

['b101']

In [33]:
han_pinyin,han_pinyin_reverse = create_han_pin_dict(entity)

  0%|          | 0/4982 [00:00<?, ?it/s]

In [11]:
dic_small = r'D:\Users\Desktop\user_s.dic'
dic_large = r'D:\Users\Desktop\user_l.dic'
with open(dic_small,'w',encoding='utf8') as f:
    f.write('\n'.join([word.lower() for word in han_pinyin_reverse.keys()]))

In [12]:
with open(r'D:\Users\Desktop\slot_pin_aug.txt','w') as f:
    f.write('\n'.join(list(han_pinyin_reverse.keys())))

In [20]:
'b101' in han_pinyin_reverse

True

In [14]:
del jieba
import jieba, re
jieba.re_han_default = re.compile("(.+)", re.U)
jieba.re_userdict = re.compile('^(.+?)(\u0040\u0040[0-9]+)?(\u0040\u0040[a-z]+)?$', re.U)
jieba.re_han_cut_all = re.compile("(.+)", re.U)
jieba.load_userdict(dic_small)


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\19052450\AppData\Local\Temp\jieba.cache
Loading model cost 0.770 seconds.
Prefix dict has been built successfully.


In [41]:
def match_pinyin(txt):
    """
    转拼音模糊匹配
    """
    match_result = []
    print(f"词槽数量:{len(han_pinyin)}")
    print(f"语音转文本后：{txt}")
    # txt_convert_dig = dig_convert(txt)
    # print(f"转大写数字后：{txt_convert_dig}")
    txt_py = py_withspace(re_sub(txt),False)[0]
    print(f"转拼音后txt_py list：{txt_py.split()}")
    # for txt_py in txt_pys:
    txt_py_list = txt_py.split()
    print(f'txt_py_list:{txt_py_list}')
    cut_words = jieba.lcut(txt_py)
    seg_result = []
    pos = 0
    for idx, word in enumerate(cut_words):
        seg_result.append((pos,word))
        pos += len(word)
    print(f"转拼音切词后cut_words：{cut_words}")
    for idx, word in enumerate(seg_result):
        pos, w = word
        if w==' ':
            continue
        if w not in txt_py_list and len(set(w.split()) & set(txt_py_list))==0:
            continue
        if w in han_pinyin_reverse:
            print(w)
            match_result.append((pos,han_pinyin_reverse[w]))
    return match_result

In [40]:
'b101' in han_pinyin_reverse

True

In [42]:
start = time()
txt = '将b1011-通风柜风量调到最大'
# txt = '氧气r,核尘是'
print(f"匹配结果:{match_pinyin(txt.lower())}")
print(f"*********耗时：{1000*(time()-start)} ms************\n")

词槽数量:4863
语音转文本后：将b1011-通风柜风量调到最大
转拼音后txt_py list：['jiang', 'b1011', 'tong', 'feng', 'gui', 'feng', 'liang', 'diao', 'dao', 'zui', 'da']
转拼音切词后cut_words：['jia', 'ng', ' ', 'b101', '1 tong feng gui', ' ', 'feng', ' ', 'li', 'an', 'g', ' ', 'diao', ' ', 'dao', ' ', 'zui', ' ', 'da']
1 tong feng gui
dao
匹配结果:[(10, ['1通风柜', '1通风柜', '1通风柜']), (42, ['氘'])]
*********耗时：4.015207290649414 ms************



In [54]:
import requests
requests.post("http://127.0.0.1:5000/update",json={"USE_PINYIN":1},headers = {'Content-Type':'application/json'}).text

'<p>1</p>'

In [153]:
py_withspace('氢氧化啦，1201',False)[0]
# jieba.lcut('qing yang hua la,1201')

'qing yang hua la ，1201'

In [2]:
num = 1
dict_url_list = []
while True:
    url = f'https://pinyin.sogou.com/dict/cate/index/113/default/{num}'
    html = requests.get(url).text
    tree = etree.HTML(html)
    for t, e in zip(tree.xpath('//div[@class="detail_title"]/a/text()'), tree.xpath('//div[@class="dict_dl_btn"]/a/@href')):
        dict_url_list.append((t, e))
    if "下一页" in html:
        num+=1
    else:
        break

In [3]:
len(dict_url_list)

68

In [6]:
# https://pinyin.sogou.com/dict/cate/index/117

idx = [119,117,103,108,109,110,111,112,114,115,117,118,123,125]
dict_url_list1= []
for idx, i in tqdm(enumerate(idx)):
    num=1
    while True:
        url = f'https://pinyin.sogou.com/dict/cate/index/{idx}/default/{num}'
        html = requests.get(url).text
        tree = etree.HTML(html)
        try:
            for t, e in zip(tree.xpath('//div[@class="detail_title"]/a/text()'), tree.xpath('//div[@class="dict_dl_btn"]/a/@href')):
                dict_url_list1.append((t, e))
        except:
            pass
        if "下一页" in html:
            num+=1
        else:
            break

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [4]:
dict_url_list[:10]

[('建筑词汇大全【官方推荐】',
  'https://pinyin.sogou.com/dict/download_cell.php?id=15118&name=%E5%BB%BA%E7%AD%91%E8%AF%8D%E6%B1%87%E5%A4%A7%E5%85%A8%E3%80%90%E5%AE%98%E6%96%B9%E6%8E%A8%E8%8D%90%E3%80%91'),
 ('建筑、结构',
  'https://pinyin.sogou.com/dict/download_cell.php?id=10217&name=%E5%BB%BA%E7%AD%91%E3%80%81%E7%BB%93%E6%9E%84'),
 ('建筑及相关专业',
  'https://pinyin.sogou.com/dict/download_cell.php?id=8618&name=%E5%BB%BA%E7%AD%91%E5%8F%8A%E7%9B%B8%E5%85%B3%E4%B8%93%E4%B8%9A'),
 ('建筑城市',
  'https://pinyin.sogou.com/dict/download_cell.php?id=152&name=%E5%BB%BA%E7%AD%91%E5%9F%8E%E5%B8%82'),
 ('建筑结构',
  'https://pinyin.sogou.com/dict/download_cell.php?id=1351&name=%E5%BB%BA%E7%AD%91%E7%BB%93%E6%9E%84'),
 ('建筑词库',
  'https://pinyin.sogou.com/dict/download_cell.php?id=603&name=%E5%BB%BA%E7%AD%91%E8%AF%8D%E5%BA%93'),
 ('建筑电气',
  'https://pinyin.sogou.com/dict/download_cell.php?id=1398&name=%E5%BB%BA%E7%AD%91%E7%94%B5%E6%B0%94'),
 ('建筑电气（住宅篇）',
  'https://pinyin.sogou.com/dict/download_cell.php?id=5664&name=%E5

In [15]:
import pandas as pd
import numpy as np

In [68]:
ques_list = []
for i in range(1,7):
    sheet_list = []
    df = pd.read_excel(r"D:\Users\Desktop\语音场景相似句.xlsx",engine='openpyxl',sheet_name=i)
    for i in range(len(df)):
        sheet_list.append([ques.split('\n')[0].strip() for ques in  df.loc[i].to_list()[0:] if type(ques)!=float])
    ques_list.append(sheet_list)

In [75]:
with open(r'D:\Users\Desktop\语音场景相似句_v1.txt','w') as f:
    for ques in ques_list: 
        # f.write()
        for q in ques:
            f.write('\n'.join(q)+'\n')
    

In [44]:
t

['哪些空调没关', '有空调没有关吗', '空调忘关的有哪些', '未关闭的空调的实验室有哪些']